In [1]:
import xmltodict
import requests
import json
import bs4

In [2]:
with open("config.json") as f:
    config = json.load(f)

In [3]:
with open("payload_data.json") as fp:
    headers = json.load(fp)

In [4]:
session = None
profile = None
def authenticate():
    global session
    global profile
    
    data =  {
        "username": config['user'],
        "password": config['passw'],
        "client": "mRIC"
    }

    auth_requests = requests.post(  "https://mlog.unitymedia.de/getprofiles/ajax-getprofiles", 
                                    data=data, 
                                    headers=headers['headersAuthentication'])

    text_dict = xmltodict.parse(auth_requests.text)
    session = text_dict['profiles']['session']
    profile =   text_dict['profiles']['profile'][-1] \
                if type(text_dict['profiles']['profile']) == list \
                else text_dict['profiles']['profile']

In [5]:
authenticate()
session, profile

('4bc16c07-b02e-49db-910c-5fcb4d9a9f3a', 'Innendienst Disponent (FS)')

In [6]:
def generate_records(order_dict, data):

    for row in data:
        idx = 0
        # print(len(row))
        for k, col in enumerate(order_dict):
            if idx >= len(row):
                # print("FIRST IF:", col, idx, k)
                order_dict[col].append('')
            elif int(row[idx]['colid']) == k+1:
                # print("SeCOND IF:", col, idx, k)
                order_dict[col].append(row[idx]['value'])
                idx += 1
            elif int(row[idx]['colid']) > k+1:
                # print("THIRD IF:", col, idx, k)
                order_dict[col].append('')

In [7]:
# def load_jms(user, session, profile):
#     with open('jms_payload.xml') as fp:
#         xml_str = fp.read()
        
#     xml = xmltodict.parse(xml_str)
#     xml_property = xml['message']['properties']['property']
#     xml_property[0]['@value'] = user
#     xml_property[1]['@value'] = session
#     xml_property[2]['@value'] = profile
    
#     return xmltodict.unparse(xml)

In [8]:
# def fetch_request(xml, query, query_type, entity, jms=True):
#     xml = xmltodict.parse(xml)
#     xml['message']['meta']['type'] = query_type
#     xml['message']['body'] = xmltodict.parse(entity)
#     if jms:
#         print("\n", xmltodict.unparse(xml), "\n")
#         url = f'https://mlog.unitymedia.de/jmsbridge/jms;{query_type}'
#         print("\n", url, "\n")
#     else:
#         url = f'https://mlog.unitymedia.de/queryExecution/queryExecution;{query}'

#     return requests.post(   url,
#                             headers=headers,
#                             data=xmltodict.unparse(xml))

In [9]:
# ## Fetch contact number

# query, query_type, entity = jms_requests.bind_email_template(row['TASKID'])
# response = self.fetch_request(query, query_type, entity)
# # print("response: ", response.content)
# response_xml = xmltodict.parse(response.text)
# sms_text = response_xml['message']['body']['entity']['field'][0]['@value']

# ## END
# print(f'\n{sms_text}')

# ## Send sms

# query, query_type, entity = jms_requests.send_sms(row['TASKID'], row['PHONE2'], sms_text)
# response = self.fetch_request(query, query_type, entity)

# ## END
# print('\n', "sms_resp:", response.text, ' sms_status:', response.status_code)

In [10]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>bindEMailTemplateRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateRequest" xmlns="">\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: $RegionContact.Contact(CONTACT).contactInformation.phone$ Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="emailTemplateId" value="44" />\n      <field name="taskId" value="9373412" />\n    </entity>\n  </body>\n</message>'.encode()

response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/bindEMailTemplateRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

In [11]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>855a4d24-175f-443e-b527-e6f6e6b17b32</id>\r\n    <correlationid>734ee24d-473e-406c-9f90-cf4fd2b6aa4c</correlationid>\r\n    <type>bindEMailTemplateResponse</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1706793932100</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body>\r\n    <entity xmlns="" class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse">\r\n      <field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche Gr\xc3\xbc\xc3\x9fe, Dein Vod

In [12]:
first_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [13]:
first_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>855a4d24-175f-443e-b527-e6f6e6b17b32</id>
<correlationid>734ee24d-473e-406c-9f90-cf4fd2b6aa4c</correlationid>
<type>bindEMailTemplateResponse</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1706793932100</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body>
<entity class="ch.logobject.mlogistics.bl.msg.BindEMailTemplateResponse" xmlns="">
<field name="bodyText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche GrÃ¼Ãe, Dein Vodafone-Team"/>
<field name="emailTemplateId" value="44"/>
<field name="html" value="false"/>
<field name="subje

In [14]:
params = {
    'timeout': '60000',
}

data = f'<?xml version="1.0" encoding="utf-8" standalone="yes"?>\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml">\n  <meta>\n    <type>sendSmsRequest</type>\n    <expectResponse>true</expectResponse>\n  </meta>\n  <properties>\n    <property type="string" name="username" value="Dobre.Vanessa" />\n    <property type="string" name="token" value="{session}" />\n    <property type="string" name="userprofile" value="{profile}" />\n    <property type="string" name="txtimeout" value="60" />\n  </properties>\n  <body>\n    <entity class="ch.logobject.mlogistics.bl.msg.SendSmsRequest" xmlns="">\n      <field name="emailTemplate" value="SMS_KUNDE_RÜCKRUFBITTE" />\n      <field name="messageText" value="Hallo Vodafone-Kund:in, leider haben wir Dich nicht erreicht. Ruf uns bitte an, um Deinen Termin mit dem Technik-Service zu vereinbaren: 0771 8006460 Vielen Dank! Freundliche Grüße, Dein Vodafone-Team" />\n      <field name="messageType" value="task" />\n      <field name="mobileNumbers" value="{{\'+4915161302414\'}}" />\n      <field name="taskId" value="9373412" />\n    </entity>\n  </body>\n</message>'.encode()

response = requests.post(
    'https://mlog.unitymedia.de/jmsbridge/jms/sendSmsRequest',
    params=params,
    headers=headers['headers'],
    data=data,
)

In [15]:
response.content

b'<?xml version="1.0" encoding="UTF-8"?>\r\n<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">\r\n  <meta>\r\n    <id>2d7d1edf-3a38-415d-8248-e9e1d6e5d7b0</id>\r\n    <correlationid>bb53dca1-20e0-4620-8947-aa02f524a19c</correlationid>\r\n    <type>ack</type>\r\n    <expiration>0</expiration>\r\n    <priority>4</priority>\r\n    <redelivered>false</redelivered>\r\n    <timestamp>1706793932826</timestamp>\r\n    <expectResponse>false</expectResponse>\r\n  </meta>\r\n  <properties />\r\n  <body />\r\n</message>\r\n\r\n'

In [16]:
second_response_soup = bs4.BeautifulSoup(response.text, features="xml")

In [17]:
second_response_soup

<?xml version="1.0" encoding="utf-8"?>
<message xmlns="http://www.logobject.ch/schema-ns/jmsoverxml" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.logobject.ch/schema-ns/jmsoverxml jmsoverxml.xsd">
<meta>
<id>2d7d1edf-3a38-415d-8248-e9e1d6e5d7b0</id>
<correlationid>bb53dca1-20e0-4620-8947-aa02f524a19c</correlationid>
<type>ack</type>
<expiration>0</expiration>
<priority>4</priority>
<redelivered>false</redelivered>
<timestamp>1706793932826</timestamp>
<expectResponse>false</expectResponse>
</meta>
<properties/>
<body/>
</message>